In [1]:
import transbigdata as tbd
import sumolib

net = sumolib.net.readNet('./testOneLine/guiyang.net.xml')
_, stops = tbd.getbusdata('贵阳', '48')

down_stops = stops.loc[stops['linename']=='48路(油榨街-碧海商业广场)']
up_stops = stops.loc[stops['linename']=='48路(碧海商业广场-油榨街)']
print(up_stops)



Obtaining city id: 贵阳success
48
48路(油榨街-碧海商业广场) success
48路(碧海商业广场-油榨街) success
   stationnames         linename         lon        lat  \
0        碧海商业广场  48路(碧海商业广场-油榨街)  106.618507  26.633165   
1    碧水云天(贵阳三中)  48路(碧海商业广场-油榨街)  106.617532  26.635421   
2        新瑞商业中心  48路(碧海商业广场-油榨街)  106.616776  26.637631   
3          黎阳家园  48路(碧海商业广场-油榨街)  106.618592  26.638369   
4        八匹马(北)  48路(碧海商业广场-油榨街)  106.620618  26.640386   
5       绿色未来(南)  48路(碧海商业广场-油榨街)  106.620059  26.645087   
6         市行政中心  48路(碧海商业广场-油榨街)  106.622817  26.648746   
7         观山湖公园  48路(碧海商业广场-油榨街)  106.629085  26.648672   
8       林城东路(西)  48路(碧海商业广场-油榨街)  106.633729  26.649160   
9     国际会议中心(北)  48路(碧海商业广场-油榨街)  106.641465  26.649428   
10       国际会议中心  48路(碧海商业广场-油榨街)  106.645745  26.644779   
11    国际会议中心(东)  48路(碧海商业广场-油榨街)  106.644446  26.640767   
12         金阳远大  48路(碧海商业广场-油榨街)  106.641245  26.632015   
13         阳关小区  48路(碧海商业广场-油榨街)  106.641296  26.627789   
14      贵阳高新区管委  48路(碧海商业广场-油榨街)  1

In [2]:
upStopNames = up_stops['stationnames']
upStopLon = up_stops['lon']
upStopLat = up_stops['lat']
downStopNames = down_stops['stationnames']
downStopLon = down_stops['lon']
downStopLat = down_stops['lat']

In [3]:
upStopCoord = []
downStopCoord = []
for i in range(len(upStopNames)):
    coord = net.convertLonLat2XY(upStopLon[i],upStopLat[i])
    upStopCoord.append(coord)
for i in range(len(downStopNames)):
    coord = net.convertLonLat2XY(downStopLon[i],downStopLat[i])
    downStopCoord.append(coord)
print(upStopNames)
print(downStopNames)

0          碧海商业广场
1      碧水云天(贵阳三中)
2          新瑞商业中心
3            黎阳家园
4          八匹马(北)
5         绿色未来(南)
6           市行政中心
7           观山湖公园
8         林城东路(西)
9       国际会议中心(北)
10         国际会议中心
11      国际会议中心(东)
12           金阳远大
13           阳关小区
14        贵阳高新区管委
15    数博大道长岭南路(中)
16        黔灵山路(西)
17            老阳关
18           保利云山
19           市北立交
20             扁井
21          省政府后门
22            市北路
23            六广门
24           云岩广场
25         喷水池(南)
26            大十字
27           钻石广场
28            甲秀楼
29             省委
30           宝山南路
31            团坡桥
32            油榨街
Name: stationnames, dtype: object
0             油榨街
1             观水路
2              省委
3             甲秀楼
4            钻石广场
5             大十字
6             喷水池
7            云岩广场
8             六广门
9             市北路
10             扁井
11           市北立交
12           保利云山
13            老阳关
14     黔灵山路(西)②分站
15    数博大道长岭南路(中)
16        贵阳高新区管委
17           阳关小区
18           金阳远大
19      国际会议中心(东)
20         国

In [4]:
import math
import traci


sumoBinary = "sumo-gui"
sumoConfig = [sumoBinary, '-c', "./testOneLine/guiyang.sumocfg"]
traci.start(sumoConfig)

upStopOnTheLane = []
downStopOnTheLane = []
upStopPos = []
downStopPos = []

traci.simulation.step()
for coord in upStopCoord:
    x = coord[0]
    y = coord[1]
    lanes = net.getNeighboringLanes(coord[0], coord[1], 20)
    if lanes:
        distanceAndLanes = sorted([(dist, lane) for lane, dist in lanes], key=lambda x: x[0])
        laneId = distanceAndLanes[0][1].getID()
        startCoord = traci.lane.getShape(laneId)[0]
        pos = math.sqrt((coord[0]-startCoord[0])**2 + (coord[1]-startCoord[1])**2)
        laneLength = traci.lane.getLength(laneId)
        if pos > laneLength and laneLength>10:
            pos = laneLength - 10
        elif pos >laneLength and laneLength<=10:
            pos = 0
        upStopOnTheLane.append(laneId)
        upStopPos.append(pos)

    else:
        print('no lanes found')

for coord in downStopCoord:
    x = coord[0]
    y = coord[1]
    lanes = net.getNeighboringLanes(coord[0], coord[1], 20)
    if lanes:
        distanceAndLanes = sorted([(dist, lane) for lane, dist in lanes], key=lambda x: x[0])
        laneId = distanceAndLanes[0][1].getID()
        startCoord = traci.lane.getShape(laneId)[0]
        pos = math.sqrt((coord[0]-startCoord[0])**2 + (coord[1]-startCoord[1])**2)
        laneLength = traci.lane.getLength(laneId)
        if pos > laneLength and laneLength>10:
            pos = laneLength - 10
        elif pos >laneLength and laneLength<=10:
            pos = 0
        downStopOnTheLane.append(laneId)
        downStopPos.append(pos)

    else:
        print('no lanes found')

print('done')

import pandas as pd
upnames = pd.DataFrame({'stopName':upStopNames})
downnames = pd.DataFrame({'stopName':downStopNames})
upnames.to_csv('./line48/upStopNames.csv', encoding='utf-8',index=False)
downnames.to_csv('./line48/downStopNames.csv', encoding='utf-8',index=False)
traci.simulation.step()

D:\tools\SUMO\tools\sumolib\net\__init__.py:312: UserWarning: Module 'rtree' not available. Using brute-force fallback.
  warnings.warn("Module 'rtree' not available. Using brute-force fallback.")


done


[]

In [5]:
import pandas as pd

updata = pd.DataFrame({'upStopOnTheLane':upStopOnTheLane, 'pos':upStopPos})
downdata = pd.DataFrame({'downStopOnTheLane':downStopOnTheLane, 'pos':downStopPos})

updata.to_csv('upStopOnTheLane.csv',encoding='utf-8',index = False)
downdata.to_csv('downStopOnTheLane.csv',encoding='utf-8',index = False)
    

In [1]:
import traci
import pandas as pd

upStopOnTheLane = pd.read_csv('upStopOnTheLane.csv',encoding='utf-8')['upStopOnTheLane']
downStopOnTheLane = pd.read_csv('downStopOnTheLane.csv',encoding='utf-8')['downStopOnTheLane']

sumoBinary = "sumo-gui"
sumoConfig = [sumoBinary, '-c', "./testOneLine/guiyang.sumocfg"]
traci.start(sumoConfig)

upRoute = []
downRoute = []
for i in range(len(upStopOnTheLane)-1):
    edges = traci.simulation.findRoute(upStopOnTheLane[i][:-2], upStopOnTheLane[i+1][:-2]).edges
    if not edges:
        print('route error')
    if i !=len(upStopOnTheLane)-2:
        upRoute += edges[:-1]
    else:
        upRoute+=edges

for i in range(len(downStopOnTheLane)-1):
    edges = traci.simulation.findRoute(downStopOnTheLane[i][:-2], downStopOnTheLane[i+1][:-2]).edges
    if not edges:
        print('route error')
    if i !=len(downStopOnTheLane)-2:
        downRoute += edges[:-1]
    else:
        downRoute+=edges

    

print('done')
traci.simulation.step()

done


[]

In [2]:
updata = pd.DataFrame({'route':upRoute})
downdata = pd.DataFrame({'route':downRoute})
updata.to_csv('upRoute.csv',encoding='utf-8', index=False)
downdata.to_csv('downRoute.csv',encoding='utf-8', index=False)

In [3]:
import pandas as pd
from xml.dom import minidom

doc1=minidom.Document()
root1=doc1.createElement("addtional")
doc1.appendChild(root1)

upnames = pd.read_csv('./line48/upStopNames.csv')['stopName']
downnames = pd.read_csv('./line48/downStopNames.csv')['stopName']
uplane = pd.read_csv('./line48/upStopOnTheLane.csv')['upStopOnTheLane']
uppos = pd.read_csv('./line48/upStopOnTheLane.csv')['pos']
downlane = pd.read_csv('./line48/downStopOnTheLane.csv')['downStopOnTheLane']
downpos = pd.read_csv('./line48/downStopOnTheLane.csv')['pos']

for i in range(len(uppos)):
    stop=doc1.createElement("busStop")
    stop.setAttribute("id","048_0_"+str(i).zfill(2))
    stop.setAttribute('name',upnames[i])
    stop.setAttribute("lane",uplane[i])
    stop.setAttribute("startPos",str(uppos[i]))
    stop.setAttribute("endPos",str(uppos[i]+10))
    stop.setAttribute("friendlyPos","True")
    stop.setAttribute("lines","048")
    stop.setAttribute("personCapacity","80")
    root1.appendChild(stop)

for i in range(len(downpos)):
    stop=doc1.createElement("busStop")
    stop.setAttribute("id","048_1_"+str(i).zfill(2))
    stop.setAttribute('name',downnames[i])
    stop.setAttribute("lane",downlane[i])
    stop.setAttribute("startPos",str(downpos[i]))
    stop.setAttribute("endPos",str(downpos[i]+10))
    stop.setAttribute("friendlyPos","True")
    stop.setAttribute("lines","048")
    stop.setAttribute("personCapacity","80")
    root1.appendChild(stop)

with open('./line48/stops_048.add.xml','w',encoding='utf-8') as f:
    doc1.writexml(f, indent='\t',addindent='\t', newl='\n', encoding='utf-8')